<a href="https://colab.research.google.com/github/noushsuon/honours-project/blob/main/BERT-Base-Cased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install datasets
#!pip install evaluate
#!pip install accelerate -U
!pip install comet_ml

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import comet_ml
from comet_ml import Experiment
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

comet_ml.init(api_key='uyaIbaxDfy7jK9j7u4KNxeKNg',project_name='comet-bert-experiment')

COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

raw_datasets = load_dataset("squad")
raw_datasets

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ), is a simple, modern stone statue of Mary. [SEP]'

In [ ]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second", #only truncates the context
        stride=stride,
        return_overflowing_tokens=True, #model will store strings ignored when truncating
        return_offsets_mapping=True, #stores the location of tokens in original text
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
#use previously defined function to preprocess training data
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset) #check the sentences have been tokenised

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

[codecarbon INFO @ 19:35:40] Energy consumed for RAM : 0.009478 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:35:40] Energy consumed for all GPUs : 0.011739 kWh. Total GPU Power : 7.928020653739783 W
[codecarbon INFO @ 19:35:40] Energy consumed for all CPUs : 0.069599 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:35:40] 0.090816 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:35:55] Energy consumed for RAM : 0.009502 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:35:55] Energy consumed for all GPUs : 0.011776 kWh. Total GPU Power : 8.98418137706868 W
[codecarbon INFO @ 19:35:55] Energy consumed for all CPUs : 0.069776 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:35:55] 0.091054 kWh of electricity used since the beginning.


(87599, 88729)

In [ ]:
'''
similar to training data except
  -without start and end positions
  -simplified offset mapping
  -uses example ids
'''
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
#use function to preprocess validation data
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

[codecarbon INFO @ 19:36:10] Energy consumed for RAM : 0.009526 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:36:10] Energy consumed for all GPUs : 0.011813 kWh. Total GPU Power : 8.746109275009927 W
[codecarbon INFO @ 19:36:10] Energy consumed for all CPUs : 0.069954 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:36:10] 0.091293 kWh of electricity used since the beginning.


(10570, 10822)

post processing

In [ ]:
#select first 100 validation
small_eval_set = raw_datasets["validation"].select(range(100))
trained_checkpoint =trained_checkpoint = "distilbert-base-cased-distilled-squad" # baseline provided by huggingface

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)

with torch.no_grad(): #disables uneccessary gradient computation
    outputs = trained_model(**batch)

[codecarbon INFO @ 19:36:25] Energy consumed for RAM : 0.009550 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:36:25] Energy consumed for all GPUs : 0.011854 kWh. Total GPU Power : 9.81594391977938 W
[codecarbon INFO @ 19:36:25] Energy consumed for all CPUs : 0.070130 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:36:25] 0.091533 kWh of electricity used since the beginning.


In [ ]:
#group features by example id in example_to_features dictionary
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)

In [ ]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [ ]:
# for each evaluation example: generate potential answers and pick best one based on logit score
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in small_eval_set:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = eval_set["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

[codecarbon INFO @ 19:36:40] Energy consumed for RAM : 0.009574 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:36:40] Energy consumed for all GPUs : 0.011922 kWh. Total GPU Power : 16.286672168549163 W
[codecarbon INFO @ 19:36:40] Energy consumed for all CPUs : 0.070307 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:36:40] 0.091803 kWh of electricity used since the beginning.


In [ ]:
import evaluate

metric = evaluate.load("squad")

In [ ]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in small_eval_set
]
print(predicted_answers[0])
print(theoretical_answers[0])

{'id': '56be4db0acb8001400a502ec', 'prediction_text': 'Denver Broncos'}
{'id': '56be4db0acb8001400a502ec', 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], 'answer_start': [177, 177, 177]}}


In [ ]:
metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 83.0, 'f1': 88.25000000000004}

# bing bing bing

In [ ]:
from tqdm.auto import tqdm

def get_example(index):
    return tokenized_datasets["test"][index]["text"]

def compute_metrics(start_logits, end_logits, features, examples):
    experiment = comet_ml.get_global_experiment()
    example_to_features = collections.defaultdict(list)

    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []

    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples
    ]
    metric_result = metric.compute(predictions=predicted_answers, references=theoretical_answers)
    if experiment:
        #experiment.set_epoch(epoch)
        experiment.log_metric("f1", metric_result['f1'])
        experiment.log_metric("exact_match", metric_result['exact_match'])
    return metric_result

In [ ]:
compute_metrics(start_logits, end_logits, eval_set, small_eval_set)

  0%|          | 0/100 [00:00<?, ?it/s]

{'exact_match': 83.0, 'f1': 88.25000000000004}

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

[codecarbon INFO @ 19:36:55] Energy consumed for RAM : 0.009598 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:36:55] Energy consumed for all GPUs : 0.011969 kWh. Total GPU Power : 11.388133656432679 W
[codecarbon INFO @ 19:36:55] Energy consumed for all CPUs : 0.070484 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:36:55] 0.092051 kWh of electricity used since the beginning.
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import transformers
from transformers import TrainingArguments
import accelerate

%env COMET_MODE=ONLINE
%env COMET_LOG_ASSETS=TRUE

args = TrainingArguments(
    "bert-finetuned-squad",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=100,
    fp16=True,
    push_to_hub=True,
    report_to=["comet_ml"],
    logging_strategy="epoch",
    seed=42,
    load_best_model_at_end=False,
    save_total_limit=1,
)

env: COMET_MODE=ONLINE
env: COMET_LOG_ASSETS=TRUE


In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
[codecarbon INFO @ 19:36:59] Energy consumed for RAM : 0.009605 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:36:59] Energy consumed for all GPUs : 0.011987 kWh. Total GPU Power : 16.328498776341462 W
[codecarbon INFO @ 19:36:59] Energy consumed for all CPUs : 0.070532 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:36:59] 0.092124 kWh of electricity used since the beginning.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ------------------------------------

Epoch,Training Loss,Validation Loss
1,1.235600,No log
2,0.743400,No log
3,0.502800,No log


[codecarbon INFO @ 19:37:18] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:37:18] Energy consumed for all GPUs : 0.000437 kWh. Total GPU Power : 104.9303895813566 W
[codecarbon INFO @ 19:37:18] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:37:18] 0.000639 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:37:33] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:37:33] Energy consumed for all GPUs : 0.000928 kWh. Total GPU Power : 117.68637030832777 W
[codecarbon INFO @ 19:37:33] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:37:33] 0.001330 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:37:48] Energy consumed for RAM : 0.000072 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:37:48] Energy consumed for all GPUs : 0.001415 kWh. Total GPU Power : 116.985974259802

TrainOutput(global_step=33276, training_loss=0.827232957875045, metrics={'train_runtime': 3375.8641, 'train_samples_per_second': 78.85, 'train_steps_per_second': 9.857, 'total_flos': 5.216534983896422e+16, 'train_loss': 0.827232957875045, 'epoch': 3.0})

In [ ]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

  0%|          | 0/10570 [00:00<?, ?it/s]

{'exact_match': 81.06906338694418, 'f1': 88.55576923641782}

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

'https://huggingface.co/noushsuon/bert-finetuned-squad/tree/main/'

In [ ]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=8
)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-squad"
repo_name = get_full_repo_name(model_name)
repo_name

'noushsuon/bert-finetuned-squad'

In [ ]:
from huggingface_hub import Repository

output_dir = "bert-finetuned-squad-accelerate"

repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/bert-finetuned-squad-accelerate is already a clone of https://huggingface.co/noushsuon/bert-finetuned-squad. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets["validation"]
    )

    for key, value in metrics.items():
        experiment.log_metric(key, value)

    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/33276 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/1353 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 0: {'exact_match': 80.84200567644277, 'f1': 88.20444834647095}
Evaluation!


...

Evaluation!


  0%|          | 0/1353 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 2: {'exact_match': 81.13528855250709, 'f1': 88.80447066512357}


Several commits (3) will be pushed upstream.


In [ ]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

In [ ]:
from transformers import pipeline

model_checkpoint = "noushsuon/bert-finetuned-squad"
question_answerer = pipeline("question-answering", model=model_checkpoint)

context = """
London is in England
"""
question = "What country is London in?"
question_answerer(question=question, context=context)

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

{'score': 0.9992902278900146, 'start': 14, 'end': 21, 'answer': 'England'}